In [12]:
import requests
from concurrent.futures import ThreadPoolExecutor
import subprocess
import time
import re
import pandas as pd
import numpy as np
import PyPDF2
import os
import ast
from tqdm import tqdm
from collections import OrderedDict
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import subprocess
import time
import undetected_chromedriver as uc
import re

In [130]:
def get_pdf_links_with_titles(file_path):
    pdf_dict = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line.count(',') < 2:
                continue  # not enough parts, skip
            
            parts = line.split(',')
            scopus_id = parts[0].strip()
            link = parts[-1].strip() if parts[-1].strip() else None
            title = ','.join(parts[1:-1]).strip()  # join all parts between scopus_id and link
            
            if link and 'ieeexplore-ieee' in link.lower() and not link.lower().endswith(".pdf"):
                pdf_dict[title] = {'scopus_id': scopus_id, 'link': link}
    
    return pdf_dict

if __name__ == "__main__":
    all_pdfs = {}
    
    file_paths = [
        # './Downloads/retrieved_links_new/retrieved_links_65000-70000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_85000-90000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_95000-96788.txt',
        # './Downloads/retrieved_links_new/retrieved_links_96789-100000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_100000-105000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_105000-110000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_110000-115000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_115000-120000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_120000-125000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_125000-127000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_127000-128000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_128000-135000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_135000-136000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_136000-138000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_138000-140000.txt'[
        # './Downloads/retrieved_links_new/retrieved_links_140000-145000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_145000-150000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_150000-155000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_155000-160000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_160000-165000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_165000-170000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_170000-175000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_175000-180000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_180000-185000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_185000-190000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_190000-195000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_195000-200000.txt',
        # './Downloads/retrieved_links_new/retrieved_links_200000-202947.txt'
        './Downloads/retrieved_links_new/retrieved_links_60000-65000.txt',
        './Downloads/retrieved_links_new/retrieved_links_70000-75000.txt',
        './Downloads/retrieved_links_new/retrieved_links_75000-80000.txt',
        './Downloads/retrieved_links_new/retrieved_links_80000-85000.txt',
        './Downloads/retrieved_links_new/retrieved_links_90000-95000.txt',
    ]
    
    for path in file_paths:
        pdf_dict = get_pdf_links_with_titles(path)
        all_pdfs.update(pdf_dict)  # Merges new dict into main one

    print(f"Total unique PDF entries: {len(all_pdfs)}")

Total unique PDF entries: 387


In [131]:
def slice_dict(d, start, end):
    return dict(list(d.items())[start:end])

In [132]:
batch = slice_dict(all_pdfs, 0, 400)
        
all_scopus_ids_in_data = {info['scopus_id'] for info in batch.values()}

remove_df = pd.read_excel('/Users/dtuser/Downloads/Remove_Scopus.xlsx')

# Extract SCOPUS IDs from Excel (assuming the column is named something like 'scopus_id')
remove_scopus_ids = set(remove_df.iloc[:, 1].astype(str).str.extract(r'SCOPUS_ID:(\d+)', expand=False).dropna())


# Check for overlap
overlap_ids = all_scopus_ids_in_data.intersection(remove_scopus_ids)

# Report result
if not overlap_ids:
    print("✅ No SCOPUS IDs in the batch are present in the Excel file.")
else:
    print(f"❌ {len(overlap_ids)} SCOPUS IDs found in both batch and Excel file.")
    print("Conflicting IDs:", overlap_ids)

✅ No SCOPUS IDs in the batch are present in the Excel file.


In [139]:
def ieee_direct_open_pdf_in_safari_and_save(pdf_url):
    apple_script = f"""
    tell application "Safari"
        activate
        open location "{pdf_url}"
        delay 5
    end tell

    tell application "Safari"
        do JavaScript "
            (function() {{
                const pdfLinks = Array.from(document.querySelectorAll('a')).filter(a =>
                    a.href.includes('/stamp/stamp.jsp') &&
                    (a.className.includes('xpl-btn-pdf') || a.className.includes('stats_PDF_'))
                );
                if (pdfLinks.length > 0) {{
                    pdfLinks[0].click();
                }} else {{
                    console.warn('No PDF link found');
                }}
            }})();
        " in document 1
        delay 5
    end tell

    -- Optional: download click can still be done using cliclick or auto-save preferences
    tell application "System Events"
        delay 15
        do shell script "cliclick c:799,829"  -- Adjust based on actual download icon location
        delay 1
    end tell

    tell application "Safari"
        activate
        tell front window
            close current tab
        end tell
    end tell
    """

    # Save and run AppleScript
    import subprocess
    subprocess.run(["osascript", "-e", apple_script])

In [70]:
# ieee_direct_open_pdf_in_safari_and_save('https://ieeexplore-ieee-org.ezproxy1.lib.asu.edu/search/searchresult.jsp?searchWithin=%22Publication%20Number%22:9813697&searchWithin=%22Document%20Title%22:Electrical%20Equivalent%20Circuit%20of%20Split%20Shaft%20Wind%20Energy%20Conversion%20Systems')

In [71]:
# ieee_direct_open_pdf_in_safari_and_save('https://ieeexplore-ieee-org.ezproxy1.lib.asu.edu/document/8391809')

In [134]:
def clear_history():
    apple_script = """
    tell application "Safari"
        activate  -- Bring Safari to the front
    end tell

    delay 2  -- Give Safari time to come forward

    tell application "System Events"
        tell process "Safari"
            set frontmost to true  -- Ensure Safari remains the active window
            delay 1
        end tell

        -- Click at position 1
        do shell script "cliclick c:73,1"
        delay 1.5

        -- Click at position 2
        do shell script "cliclick c:115,196"
        delay 1.5

        -- Click at position 3
        do shell script "cliclick c:865,509"
        do shell script "cliclick c:865,509"
        do shell script "cliclick c:846,350"
        do shell script "cliclick c:846,350"
        delay 3
    end tell
    """

    import subprocess
    subprocess.run(["osascript", "-e", apple_script], text=True)

In [135]:
def new_tab():
    apple_script = """
    tell application "Safari"
        activate
        if not (exists window 1) then
            make new document
        else
            tell window 1
                make new tab
            end tell
        end if
    end tell
    """
    import subprocess
    subprocess.run(["osascript", "-e", apple_script], text=True)

In [141]:
batch = slice_dict(all_pdfs, 0, 400)

folder_path = '/Users/dtuser/Downloads/onlinewiley_pdfs_10Jul'

scopus_ids_in_folder = set()
for filename in os.listdir(folder_path):
    if '-' in filename:
        try:
            scopus_id_part = filename.rsplit('-', 1)[-1].split('.')[0]
            scopus_ids_in_folder.add(scopus_id_part)
        except IndexError:
            continue
    else:
        try:
            if int(filename[:-4]):
                scopus_id_part = filename[:-4]
                scopus_ids_in_folder.add(scopus_id_part)
        except:
            continue
        
all_scopus_ids_in_data = {info['scopus_id'] for info in batch.values()}

missing_scopus_ids = all_scopus_ids_in_data - scopus_ids_in_folder

missing_entries = {title: info for title, info in batch.items() if info['scopus_id'] in missing_scopus_ids}
len(scopus_ids_in_folder), len(missing_entries)

(757, 77)

In [142]:
download_path = '/Users/dtuser/Downloads/onlinewiley_pdfs_10Jul/'
count = 0
clear_history()
for title, info in missing_entries.items():
    count += 1
    if count % 25 == 0:
        clear_history()
    if count % 30 == 0:
        new_tab()
        new_tab()
        new_tab()
    link = info['link']
    scopus_id = info['scopus_id']
    try:
        ieee_direct_open_pdf_in_safari_and_save(link)
        time.sleep(2)
        files = os.listdir('./Downloads/')
        pdf_files = [f for f in files if f.endswith(".pdf")]
        current_time = time.time()
        recent_files = [
            os.path.join('', f)
            for f in pdf_files
            if current_time - os.path.getctime(os.path.join('./Downloads/', f)) <= 10
        ]
        
        if recent_files == []:
            continue
        safe_title = re.sub(r'</?inf>', '', title)
        safe_title = re.sub(r'[^A-Za-z0-9 ]+', '', safe_title)
        safe_title = safe_title.strip() + '-' + scopus_id
        new_name = os.path.join(download_path, f"{safe_title}.pdf")  # Rename with title
        latest_file = max([os.path.join('./Downloads', f) for f in recent_files], key=os.path.getctime)
        try:
            os.rename(latest_file, new_name)
            print(f"📂 Renamed file to: {new_name}")
        except OSError as e:
            if e.errno == 63:  
                fallback_name = os.path.join(download_path, f"{scopus_id}.pdf")
                os.rename(latest_file, fallback_name)
                print(f"⚠️ Filename too long, saved as: {fallback_name}")
            else:
                raise e  # Reraise other unexpected errors
        continue
    except Exception as e:
        print(f"❌ Error processing ScienceDirect PDF: {e}")
        continue   


KeyboardInterrupt: 